# Introdução à Ciência de Dados - Projeto 1

<p align="justify">
Nesse proejto iremos realizar uma análise de dad os sobre um dataset de carros usados da cidade de Recife - PE. O dataset foi montado a partir de dados coletados por webscraping de diversos sites de vendas. O dataset contém informações sobre o preço, ano, quilometragem, marca, modelo, tipo de combustível, tipo de câmbio e cor dos veículos. O objetivo desse projeto é realizar uma análise exploratória dos dados, respondendo a algumas perguntas sobre o dataset e gerando visualizações que possam ajudar a entender melhor o dataset.

### Equipe:
> Pedro Henrique Almeida Girão Peixinho (phagp)

> Victor Gabriel de Carvalho (vgc3)

### Tópicos Avançados em Gerenciamento de Dados e Informação IF697 - 2024.1 - Centro de Informáica UFPE

## Imports e Configurações

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

from fancyimpute import KNN

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# from google.colab import drive
# drive.mount('/content/drive')

Aqui está o dataset montado, ele é composto por 1165 amostras com 11 features cada.

In [2]:
df_cars = pd.read_csv('carscraper\cars.csv')
df_cars.shape

(1124, 12)

---

## Pré-processamento dos Dados

<p align="justify">
Nessa etapa, será feita a leitura do dataset e a análise inicial dos dados. Mais precisamente, focaremos na definição e correção de tipos de dados, tratamento de valores faltantes, mormalização e discretização de dados, e limpeza geral.

<p align="justify">
Abaixo está a aparência inicial do dataset, após a leitura dos dados. Veja que algumas colunas não apresentam valores estatísticos significantes para a análise, como a coluna de "page" e a de "car_desc" (que ja teve seus valores diluídos entre outras colunas, como a "car_engine" e "car_gearbox"). Elas serão removidas do dataset.

In [3]:
df_cars.head()

,page,car_brand,car_name,car_price,car_km,car_year,car_desc,car_store,car_engine,car_gearbox,car_fuel,car_color
0,https://grupoautonunes.com/estoque/?zero_km=0&...,Fiat,500E,125990.0,7521.0,2022,ICON ELÉTRICO,NaN,Elétrico,Automatico,Elétrico,NaN
1,https://grupoautonunes.com/estoque/?zero_km=0&...,Audi,Q3,128980.0,40000.0,2018,1.4 TFSI AMBIENTE FLEX 4P S TRONIC,NaN,1.4,Automatico,Flex,NaN
2,https://grupoautonunes.com/estoque/?zero_km=0&...,Chevrolet,Onix,70990.0,40686.0,2023,1.0 FLEX LT MANUAL,NaN,1.0,Manual,Flex,NaN
3,https://grupoautonunes.com/estoque/?zero_km=0&...,Chevrolet,Onix,79990.0,57643.0,2022,1.0 TURBO FLEX PLUS LTZ AUTOMÁTICO,NaN,1.0 Turbo,Automatico,Flex,NaN
4,https://grupoautonunes.com/estoque/?zero_km=0&...,Volkswagen,Saveiro,77990.0,29000.0,2023,1.6 MSI TRENDLINE CS 8V FLEX 2P MANUAL,NaN,1.6,Manual,Flex,NaN


<p align="justify">
Além das colunas irrelevantes, será feita a remoção de linhas duplicadas, caso existam.

In [4]:
df_cars.drop(['page', 'car_name', 'car_desc'], axis=1, inplace=True)
df_cars.drop_duplicates(inplace=True)

df_cars.shape

(1103, 9)

<p align="justify">
Agora podemos fazer a primeira análise dos dados numéricos do dataset.

In [5]:
df_cars.describe()

,car_price,car_km,car_year
count,1103.000000,1103.000000,1103.000000
mean,107248.161378,45323.883953,2020.113327
std,104532.480654,33650.024486,3.032617
min,24990.000000,0.000000,2000.000000
25%,59900.000000,25113.500000,2019.000000
50%,76990.000000,43339.000000,2021.000000
75%,109900.000000,61505.000000,2022.000000
max,968900.000000,654000.000000,2025.000000


---

### Definição de Tipos

<p align="justify">
Agora iremos fazer o tratamento dos tipos de dados do dataset. Isso é importante para carantir a correture das operações estatísticas e visualizações que iremos realizar sobre os dados. Apesar do crawler ter feito um bom trabalho em relação aos tipos de dados, ainda existem algumas colunas que precisam ser ajustadas.

In [6]:
df_cars.dtypes

car_brand       object
car_price      float64
car_km         float64
car_year         int64
car_store       object
car_engine      object
car_gearbox     object
car_fuel        object
car_color       object
dtype: object

<p align="justify">
A primeira coisa a ser é mudar o tipo de dados de "object" para "category", para as colunas que representam categorias. Isso é importante para facilitar a manipulação desses dados e possibilitar uma série de operações nativas do pandas.

In [7]:
df_cars['car_brand'] = df_cars['car_brand'].astype('category')
df_cars['car_engine'] = df_cars['car_engine'].astype('category')
df_cars['car_gearbox'] = df_cars['car_gearbox'].astype('category')
df_cars['car_fuel'] = df_cars['car_fuel'].astype('category')
df_cars['car_color'] = df_cars['car_color'].astype('category')
df_cars['car_store'] = df_cars['car_store'].astype('category')

df_categories = df_cars.copy()

df_cars.dtypes

car_brand      category
car_price       float64
car_km          float64
car_year          int64
car_store      category
car_engine     category
car_gearbox    category
car_fuel       category
car_color      category
dtype: object

<p align="justify">
Uma das possibilidades ao se usar o tipo categórico é a visualização de todas as categorias por feature, como é mostrado abaixo.

In [8]:
print(df_cars['car_brand'].cat.categories)
print(df_cars['car_engine'].cat.categories)
print(df_cars['car_gearbox'].cat.categories)
print(df_cars['car_fuel'].cat.categories)
print(df_cars['car_color'].cat.categories)
print(df_cars['car_store'].cat.categories)

Index(['Alfa', 'Audi', 'Bmw', 'Byd', 'Caoa', 'Chery', 'Chevrolet', 'Citroën',
       'Fiat', 'Ford', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Kia', 'Land',
       'Lexus', 'Mercedes-Benz', 'Mini', 'Mitsubishi', 'Nissan', 'Peugeot',
       'Porsche', 'Ram', 'Renault', 'Smart', 'Suzuki', 'Toyota', 'Troller',
       'Volkswagen', 'Volvo'],
      dtype='object')
Index(['1.0', '1.0 Turbo', '1.2', '1.2 Turbo', '1.3', '1.3 Turbo', '1.4',
       '1.4 Turbo', '1.5', '1.5 Turbo', '1.6', '1.6 Turbo', '1.8', '2.0',
       '2.0 Turbo', '2.3', '2.4', '2.5', '2.7', '2.8', '2.8 Turbo', '2.9',
       '3.0', '3.0 Turbo', '3.2', '3.2 Turbo', '3.6', '4.0', '6.7 Turbo',
       'Elétrico'],
      dtype='object')
Index(['Automatico', 'Manual'], dtype='object')
Index(['Diesel', 'Elétrico', 'Flex', 'Gasolina', 'Gnv', 'Híbrido'], dtype='object')
Index(['Azul', 'Branco', 'Cinza', 'Dourado', 'Laranja', 'Prata', 'Preto',
       'Verde', 'Vermelho'],
      dtype='object')
Index([' Pedragon Afogados ', ' Pedragon Av N

---

### Tratamento de Dados Ausentes

<p align="justify">
Com os tipos definidos, podemos agora analisar a presença de valores ausentes no dataset. Isso é importante para garantir a qualidade dos dados e evitar erros em operações estatísticas e visualizações. É muito comum, em datasets feios por webscraping, a presença de valores ausentes, então é importante lidar com eles de forma adequada.

In [9]:
df_cars.isnull().sum()

car_brand         0
car_price         0
car_km            0
car_year          0
car_store      1095
car_engine        0
car_gearbox       0
car_fuel         48
car_color       986
dtype: int64

<p align="justify">
A primeira coisa que faremos é a remoção de colunas com mais de 70% dos valores ausentes. Outros métodos de inferêncoa e adição de valores seria inviável, pois a quantidade de valores ausentes é muito grande.

In [10]:
df_cars.dropna(thresh=0.7*len(df_cars), axis=1, inplace=True)
df_cars.columns

Index(['car_brand', 'car_price', 'car_km', 'car_year', 'car_engine',
       'car_gearbox', 'car_fuel'],
      dtype='object')

<p align="justify">
Como era de se esperar, as colunas "car_store" e "car_color" foram removidas. Agora podemos analisar a presença de valores ausentes na coluna "car_engine" e "car_fuel", as única que ainda tem valores falntes.

In [11]:
df_cars.count()

car_brand      1103
car_price      1103
car_km         1103
car_year       1103
car_engine     1103
car_gearbox    1103
car_fuel       1055
dtype: int64

<p align="justify">
Para fazer os processos de adição de valores faltantes, é importante converter os campos categóricos para um formato numérico. Dessa forma, podemos fazer operações como média, KNN etc.

In [12]:
df_cars.dtypes

car_brand      category
car_price       float64
car_km          float64
car_year          int64
car_engine     category
car_gearbox    category
car_fuel       category
dtype: object

In [13]:
df_cars['car_brand'] = df_cars['car_brand'].cat.codes
df_cars['car_engine'] = df_cars['car_engine'].cat.codes
df_cars['car_gearbox'] = df_cars['car_gearbox'].cat.codes
df_cars['car_fuel'] = df_cars['car_fuel'].cat.codes.replace(-1, np.nan)

df_cars.count()

car_brand      1103
car_price      1103
car_km         1103
car_year       1103
car_engine     1103
car_gearbox    1103
car_fuel       1055
dtype: int64

In [14]:
print('Brand: ', df_cars['car_brand'].mean(), df_cars['car_brand'].median())
print('Engine: ', df_cars['car_engine'].mean(), df_cars['car_engine'].median())
print('Gearbox: ', df_cars['car_gearbox'].mean(), df_cars['car_gearbox'].median())
print('Fuel: ', df_cars['car_fuel'].mean(), df_cars['car_fuel'].median())

Brand:  14.053490480507707 11.0
Engine:  6.647325475974615 6.0
Gearbox:  0.45602901178603805 0.0
Fuel:  2.0113744075829385 2.0


Utilizando Média e Mediana

In [15]:
df_mean_fill = df_cars.copy()

df_mean_fill['car_fuel'] = df_mean_fill['car_fuel'].fillna(df_mean_fill['car_fuel'].mean())
df_mean_fill.isnull().sum()

car_brand      0
car_price      0
car_km         0
car_year       0
car_engine     0
car_gearbox    0
car_fuel       0
dtype: int64

In [16]:
print('Fuel: ', df_mean_fill['car_fuel'].mean(), df_mean_fill['car_fuel'].median())

Fuel:  2.011374407582938 2.0


In [17]:
df_median_fill = df_cars.copy()

df_median_fill['car_fuel'] = df_median_fill['car_fuel'].fillna(df_median_fill['car_fuel'].median())
df_median_fill.isnull().sum()

car_brand      0
car_price      0
car_km         0
car_year       0
car_engine     0
car_gearbox    0
car_fuel       0
dtype: int64

In [18]:
print('Fuel: ', df_median_fill['car_fuel'].mean(), df_median_fill['car_fuel'].median())

Fuel:  2.0108794197642794 2.0


Utilizando KNN

In [19]:
df_knn = KNN(k=5).fit_transform(df_cars)
df_knn.shape

Imputing row 1/1103 with 0 missing, elapsed time: 0.172
Imputing row 101/1103 with 0 missing, elapsed time: 0.172
Imputing row 201/1103 with 0 missing, elapsed time: 0.172
Imputing row 301/1103 with 1 missing, elapsed time: 0.173
Imputing row 401/1103 with 0 missing, elapsed time: 0.174
Imputing row 501/1103 with 0 missing, elapsed time: 0.174
Imputing row 601/1103 with 0 missing, elapsed time: 0.174
Imputing row 701/1103 with 0 missing, elapsed time: 0.175
Imputing row 801/1103 with 0 missing, elapsed time: 0.175
Imputing row 901/1103 with 0 missing, elapsed time: 0.175
Imputing row 1001/1103 with 0 missing, elapsed time: 0.175
Imputing row 1101/1103 with 0 missing, elapsed time: 0.175


(1103, 7)

In [20]:
df_knn_fill = pd.DataFrame(data=df_knn[0:,0:],columns=[df_cars.columns])
df_knn_fill.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel
0,8.0,125990.0,7521.0,2022.0,29.0,0.0,1.0
1,1.0,128980.0,40000.0,2018.0,6.0,0.0,2.0
2,6.0,70990.0,40686.0,2023.0,0.0,1.0,2.0
3,6.0,79990.0,57643.0,2022.0,1.0,0.0,2.0
4,29.0,77990.0,29000.0,2023.0,10.0,1.0,2.0


In [21]:
df_knn_fill.isnull().sum()

car_brand      0
car_price      0
car_km         0
car_year       0
car_engine     0
car_gearbox    0
car_fuel       0
dtype: int64

In [22]:
print('Fuel: ', df_knn_fill['car_fuel'].mean(), df_knn_fill['car_fuel'].median())

Fuel:  car_fuel    2.012581
dtype: float64 car_fuel    2.0
dtype: float64


In [23]:
df_knn_fill.describe()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel
count,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000
mean,14.053490,107248.161378,45323.883953,2020.113327,6.647325,0.456029,2.012581
std,8.661922,104532.480654,33650.024486,3.032617,6.548008,0.498289,0.727666
min,0.000000,24990.000000,0.000000,2000.000000,0.000000,0.000000,0.000000
25%,6.000000,59900.000000,25113.500000,2019.000000,0.000000,0.000000,2.000000
50%,11.000000,76990.000000,43339.000000,2021.000000,6.000000,0.000000,2.000000
75%,24.000000,109900.000000,61505.000000,2022.000000,12.000000,1.000000,2.000000
max,30.000000,968900.000000,654000.000000,2025.000000,29.000000,1.000000,5.000000


<p align="justify">
A partir de agora, usaremos o dataset preenchido com o KNN como o dataset principal, para as anlaíses posteriores.

---

### Normalização e Discretização

<p align="justify">
O próximo passo é fazer a normalização dos dados numéricos do dataset. A normalização é importante para garantir que as features tenham a mesma escala, evitando que features com valores maiores dominem o resultado das operações. Utilizaremos tano o método de normalização Min-Max quanto o método de normalização padrão (removendo a média e escalando para variância unitária), ambos da biblioteca scikit-learn.

In [24]:
df_knn_fill.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel
0,8.0,125990.0,7521.0,2022.0,29.0,0.0,1.0
1,1.0,128980.0,40000.0,2018.0,6.0,0.0,2.0
2,6.0,70990.0,40686.0,2023.0,0.0,1.0,2.0
3,6.0,79990.0,57643.0,2022.0,1.0,0.0,2.0
4,29.0,77990.0,29000.0,2023.0,10.0,1.0,2.0


In [25]:
min_max_scaler = MinMaxScaler()
df_knn_fill_norm = min_max_scaler.fit_transform(df_knn_fill)
df_knn_fill_norm = pd.DataFrame(df_knn_fill_norm, columns=df_knn_fill.columns)
df_knn_fill_norm.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel
0,0.266667,0.107002,0.011500,0.88,1.000000,0.0,0.2
1,0.033333,0.110169,0.061162,0.72,0.206897,0.0,0.4
2,0.200000,0.048733,0.062211,0.92,0.000000,1.0,0.4
3,0.200000,0.058268,0.088139,0.88,0.034483,0.0,0.4
4,0.966667,0.056149,0.044343,0.92,0.344828,1.0,0.4


In [26]:
standard_scaler = StandardScaler()
df_knn_fill_std = standard_scaler.fit_transform(df_knn_fill)
df_knn_fill_std = pd.DataFrame(df_knn_fill_std, columns=df_knn_fill.columns)
df_knn_fill_std.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel
0,-0.699179,0.179373,-1.123923,0.622409,3.415208,-0.915605,-1.392177
1,-1.507681,0.207990,-0.158285,-0.697182,-0.098903,-0.915605,-0.017297
2,-0.930180,-0.347018,-0.137890,0.952307,-1.015628,1.092173,-0.017297
3,-0.930180,-0.260881,0.366261,0.622409,-0.862841,-0.915605,-0.017297
4,1.726325,-0.280022,-0.485328,0.952307,0.512247,1.092173,-0.017297


In [27]:
robust_scaler = RobustScaler()
df_knn_fill_robust = robust_scaler.fit_transform(df_knn_fill)
df_knn_fill_robust = pd.DataFrame(df_knn_fill_robust, columns=df_knn_fill.columns)
df_knn_fill_robust.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel
0,-0.166667,0.9800,-0.984241,0.333333,1.916667,0.0,-1.0
1,-0.555556,1.0398,-0.091752,-1.000000,0.000000,0.0,0.0
2,-0.277778,-0.1200,-0.072902,0.666667,-0.500000,1.0,0.0
3,-0.277778,0.0600,0.393059,0.333333,-0.416667,0.0,0.0
4,1.000000,0.0200,-0.394021,0.666667,0.333333,1.0,0.0


---

### Limpeza de Dados

In [28]:
fig = make_subplots(rows=1, cols=3)

fig.append_trace(go.Box(y=df_cars['car_price'], name='Price', quartilemethod="linear"), row=1, col=1)
fig.append_trace(go.Box(y=df_cars['car_km'], name='Mileage', quartilemethod="linear"), row=1, col=2)
fig.append_trace(go.Box(y=df_cars['car_year'], name='Release Year', quartilemethod="linear"), row=1, col=3)

fig.update_traces(boxpoints='all', jitter=0.3)
fig.update_layout(height=500, width=970, title_text="Numerical Columns Outliers")

#### Correlação dos dados

In [29]:
df_corr = df_cars.corr()

fig = go.Figure(
    go.Heatmap(
        x = df_corr.columns,
        y = df_corr.index,
        z = np.array(df_corr),
        text=df_corr.values,
        texttemplate='%{text:.2f}'
    )
)

fig.update_layout(height=900, width=970, title_text="Correlation Matrix")
fig.show()

#### Método de Tukey (univariado)

Calcula o IQR para cada coluna

In [30]:
q1 = df_cars.quantile(q=0.25)
q3 = df_cars.quantile(q=0.75)
iqr = q3 - q1
iqr

car_brand         18.0
car_price      50000.0
car_km         36391.5
car_year           3.0
car_engine        12.0
car_gearbox        1.0
car_fuel           0.0
dtype: float64

Remove outliers com base no preço e kilometragem do carro

In [31]:
df_cars_tukey = df_cars[(df_cars['car_price'] > (q1['car_price']-1.5*iqr['car_price'])) & (df_cars['car_price'] < (q3['car_price']+1.5*iqr['car_price']))]
df_cars_tukey = df_cars_tukey[(df_cars_tukey['car_km'] > (q1['car_km']-1.5*iqr['car_km'])) & (df_cars_tukey['car_km'] < (q3['car_km']+1.5*iqr['car_km']))]
len(df_cars_tukey)

981

In [32]:
df_cars_tukey.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel
0,8,125990.0,7521.0,2022,29,0,1.0
1,1,128980.0,40000.0,2018,6,0,2.0
2,6,70990.0,40686.0,2023,0,1,2.0
3,6,79990.0,57643.0,2022,1,0,2.0
4,29,77990.0,29000.0,2023,10,1,2.0


#### Razão de Duas Variáveis (bivariado)

In [33]:
# df_cars.plot.scatter(x='car_price',y='car_km')
fig = go.Figure(
    go.Scatter(
        x=df_categories['car_price'], 
        y=df_categories['car_km'],
        mode='markers',
    )
)

fig.update_layout(height=500, width=970, title_text="Scatter Plot of car_price x car_km")
fig.show()

In [34]:
df_cars_2_var = df_cars.copy()
df_cars_2_var['car_price_per_km'] = df_cars_2_var['car_price'] / df_cars_2_var['car_km']
df_cars_2_var.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel,car_price_per_km
0,8,125990.0,7521.0,2022,29,0,1.0,16.751762
1,1,128980.0,40000.0,2018,6,0,2.0,3.224500
2,6,70990.0,40686.0,2023,0,1,2.0,1.744826
3,6,79990.0,57643.0,2022,1,0,2.0,1.387679
4,29,77990.0,29000.0,2023,10,1,2.0,2.689310


In [35]:
fig = go.Figure(
    go.Box(y=df_cars_2_var['car_price_per_km'], name='Price per km', quartilemethod="linear")
)

fig.update_layout(height=500, width=970, title_text="Box Plot of car price per km")
fig.show()

In [36]:
fig = go.Figure(
    go.Histogram(x=df_cars_2_var['car_price_per_km'], name='Price per km')
)

fig.update_layout(height=500, width=970, title_text="Histogram of car price per km")
fig.show()


In [37]:
df_cars_2_var['car_price_per_km_log'] = np.log10(df_cars_2_var['car_price_per_km'])

fig = go.Figure(
    go.Histogram(x=df_cars_2_var['car_price_per_km_log'], name='Log of price per km')
)

fig.update_layout(height=500, width=970, title_text="Histogram of the log of car price per km")
fig.show()

In [38]:
mad = abs(df_cars_2_var['car_price_per_km_log'] - df_cars_2_var['car_price_per_km_log'].median()).median()*(1/0.6745)
print(mad)

0.3994815643854331


In [39]:
fig = go.Figure(
    go.Histogram(x=abs(df_cars_2_var['car_price_per_km_log']-df_cars_2_var['car_price_per_km_log'].median())/mad, name='Log of price per km')
)

fig.update_layout(height=500, width=970, title_text="Histogram of the log of car price per km")
fig.show()

In [40]:
len(df_cars_2_var[abs(df_cars_2_var['car_price_per_km_log']-df_cars_2_var['car_price_per_km_log'].median())/mad > 3.5])

101

In [41]:
df_cars_2_var = df_cars_2_var[abs(df_cars_2_var['car_price_per_km_log']-df_cars_2_var['car_price_per_km_log'].median())/mad < 3.5]
len(df_cars_2_var)

1002

#### Elliptic Envelope (multivariado)

In [42]:
df_knn_fill_ee = df_knn_fill.copy()

In [43]:
detector = EllipticEnvelope(contamination=0.01)
detector.fit(df_knn_fill_ee)

scores = detector.predict(df_knn_fill_ee)
df_knn_fill_ee['outlier'] = scores

df_knn_fill_ee.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel,outlier
0,8.0,125990.0,7521.0,2022.0,29.0,0.0,1.0,1
1,1.0,128980.0,40000.0,2018.0,6.0,0.0,2.0,1
2,6.0,70990.0,40686.0,2023.0,0.0,1.0,2.0,1
3,6.0,79990.0,57643.0,2022.0,1.0,0.0,2.0,1
4,29.0,77990.0,29000.0,2023.0,10.0,1.0,2.0,1


In [44]:
df_knn_fill_ee = df_knn_fill_ee[df_knn_fill_ee['outlier'] != -1]
df_knn_fill_ee.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel,outlier
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
df_knn_fill_ee.count()

car_brand      1103
car_price      1103
car_km         1103
car_year       1103
car_engine     1103
car_gearbox    1103
car_fuel       1103
outlier        1103
dtype: int64

#### Isolation Forests (multivariado)

In [ ]:
df_knn_fill_if = df_knn_fill.copy()

In [ ]:
clf = IsolationForest(max_samples=100, random_state=np.random.RandomState(42))
clf.fit(df_knn_fill_if)

scores = clf.predict(df_knn_fill_if)
df_knn_fill_if['outlier'] = scores

df_knn_fill_if.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel,outlier
0,8.0,125990.0,7521.0,2022.0,29.0,0.0,1.0,-1
1,1.0,128980.0,40000.0,2018.0,6.0,0.0,2.0,1
2,6.0,70990.0,40686.0,2023.0,0.0,1.0,2.0,1
3,6.0,79990.0,57643.0,2022.0,1.0,0.0,2.0,1
4,29.0,77990.0,29000.0,2023.0,10.0,1.0,2.0,-1


In [ ]:
df_knn_fill_if = df_knn_fill_if[df_knn_fill_if['outlier'] != -1]
df_knn_fill_if.head()

,car_brand,car_price,car_km,car_year,car_engine,car_gearbox,car_fuel,outlier
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [524]:
df_knn_fill_if.count()

car_brand        0
car_price        0
car_km           0
car_year         0
car_engine       0
car_gearbox      0
car_fuel         0
outlier        836
dtype: int64

---

### Estatísticas Descritivas

In [595]:
fig = make_subplots(rows=3, cols=1)

fig.append_trace(go.Histogram(x=df_cars['car_price'], name='Price'), row=1, col=1)
fig.append_trace(go.Histogram(x=df_cars['car_km'], name='Mileage'), row=2, col=1)
fig.append_trace(go.Histogram(x=df_cars['car_year'], name='Release Year'), row=3, col=1)

fig.update_layout(height=900, width=970, title_text="Numerical Columns Histogram")
fig.show()

Mostrar histograma dos motores (utilizando dataframe antes de utilizar cat codes)

In [66]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(x=df_categories['car_engine'])
)

fig.update_layout(height=900, width=970, title_text="Engine Histogram")
fig.show()

---

### Teste de Hipóteses

---

## Referências

- https://plotly.com/python/histograms/
- https://plotly.com/python/legend/